# VariantSpark-k Small Parquet

Smallest production-sized job (5 GB) - takes < 30 minutes to run w/ 4 qty of r4.4xlarge EC2 instance & Kubernetes

### Parameters
 - input bucket location - CSIRO S3 bucket (Sydney)
 - input files `data_s5000_v2500000...` and fc `resp`
 - number of Spark executors 56 w/ 7 GB RAM each
 - number of RF trees 500, w/ batch of 28
 - file type `-it parquet`
 - default `mtry`
 - calculate OOB as `-ro` paramater is configured w/100 trees OOB was .50 :-(

In [1]:
%%bash

set -e

MASTER=https://kubernetes.default.svc:443
INPUT_BUCKET=variant-spark

function fatal_error () {
	echo "ERROR: $1" 1>&2
	exit 1
}

if [ -z ${MASTER+x} ];
    then
        echo "You must set the MASTER environment variable to a kubernetes API endpoint";
        echo "Example: https://ABC.sk1.us-west-2.eks.amazonaws.com:443"
        exit 1
fi

if [ -z ${INPUT_BUCKET+x} ];
    then
        echo "You must set the INPUT_BUCKET environment variable to a bucket containing input data";
        echo "Example: variant-spark-k-storage"
        exit 1
fi

[[ $(type -P "spark-submit") ]] || fatal_error  "\`spark-submit\` cannot be found. Please make sure it's on your PATH."

spark-submit \
    --class au.csiro.variantspark.cli.VariantSparkApp \
    --driver-class-path ./conf \
    --master k8s://${MASTER} \
    --deploy-mode cluster \
    --name VariantSparkSmallParquet \
    --conf spark.kubernetes.authenticate.driver.serviceAccountName=spark \
    --conf spark.executor.instances=56 \
    --conf spark.executor.memory=8g \
    --conf spark.kubernetes.container.image=jamesrcounts/variantspark:002 \
    --jars http://central.maven.org/maven2/org/apache/hadoop/hadoop-aws/2.7.3/hadoop-aws-2.7.3.jar,http://central.maven.org/maven2/com/amazonaws/aws-java-sdk/1.7.4/aws-java-sdk-1.7.4.jar,http://central.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.jar \
    local:///opt/spark/jars/variant-spark_2.11-0.2.0-SNAPSHOT-all.jar importance \
        -if s3a://${INPUT_BUCKET}/datasets/synthetic/data_s5000_v2500000_r13.parquet \
        -ff s3a://${INPUT_BUCKET}/datasets/synthetic/labels_s5000_v2500000_r13_f0.125.csv \
        -fc resp \
        -it parquet \
        -v \
        -rn 500 \
        -rbs 28 \
        -ro "$@"


2018-09-15 13:18:26 WARN  DriverServiceBootstrapStep:66 - Driver's hostname would preferably be variantsparksmallparquet-105ebfa19ea83d53a5e5168d2d433976-driver-svc, but this is too long (must be <= 63 characters). Falling back to use spark-1537017506415-driver-svc as the driver service's name.
2018-09-15 13:18:27 INFO  LoggingPodStatusWatcherImpl:54 - State changed, new state: 
	 pod name: variantsparksmallparquet-105ebfa19ea83d53a5e5168d2d433976-driver
	 namespace: default
	 labels: spark-app-selector -> spark-33b2fe99628345d0b5539a8dd80249c4, spark-role -> driver
	 pod uid: d4c17cc3-b8e9-11e8-b937-0693851c96ec
	 creation time: 2018-09-15T13:18:27Z
	 service account name: spark
	 volumes: spark-init-properties, download-jars-volume, download-files-volume, spark-token-wh6vp
	 node name: N/A
	 start time: N/A
	 container images: N/A
	 phase: Pending
	 status: []
2018-09-15 13:18:27 INFO  LoggingPodStatusWatcherImpl:54 - State changed, new state: 
	 pod name: variantsparksmallparquet-10